In [ ]:
#imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
# constants
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {'Content-Type': 'application/json'}
MODEL = "llama3.2:latest"

In [ ]:
# create a messages list using the same format that we used for OpenAI
messages = [
    {
        "role": "user",
        "content": "Describe some of the business applications of Generative AI",
    }
]

In [ ]:
payload = {
    "model": MODEL,
    "messages": messages,
    "stream": False,
}

In [ ]:
# making sure the model is loaded
!ollama pull $MODEL

In [ ]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

In [ ]:
# Intoducing ollama package
import ollama
response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])  

In [ ]:
# alternative using openai python library to connect to ollama
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key="ollama")

response = ollama_via_openai.chat.completions.create(model=MODEL, messages=messages)

print(response.choices[0].message.content)

In [ ]:
import ollama
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

# constants
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {'Content-Type': 'application/json'}
MODEL = "llama3.2:latest"

# some websites you need a proper header when fetching them
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrevelant in soup(["script", "style", "img", "input"]):
            irrevelant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

system_prompt = """
Your are an asistant that analyzes the content of a webpage and provides a short 
summary, ignoreing text that might be navigation related. Respond in markdown."""

def user_prompt_for(website):
    user_promt = f"You are looking at a website titled {website.title}"
    user_promt += "\nThe contents of this website is as follows;\
        please provide short summary of this website in markdown. \
            If it includes news or announcements, please include that as well."
    user_promt += website.text
    return user_promt

def message_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

def summarize(url):
    website = Website(url)
    response = ollama.chat(model=MODEL, messages=message_for(website))
    return response.message.content

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

display_summary("https://edwarddonner.com")